In [ ]:
from google.colab import files
uploaded = files.upload()

Saving 2025-Beyond  ADMISSIONS  FREQUENTLY ASKED INQUIRY QUESTIONS TEMPLATES 1.docx to 2025-Beyond  ADMISSIONS  FREQUENTLY ASKED INQUIRY QUESTIONS TEMPLATES 1.docx


In [ ]:
!pip -q install python-docx rank-bm25 requests

import os
import re
import requests
from docx import Document
from rank_bm25 import BM25Okapi

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 253.0/253.0 kB 15.6 MB/s eta 0:00:00


In [ ]:
DOCX_PATH = "2025-Beyond  ADMISSIONS  FREQUENTLY ASKED INQUIRY QUESTIONS TEMPLATES 1.docx"

doc = Document(DOCX_PATH)

print("DOCX loaded successfully")
print("Total paragraphs in document:", len(doc.paragraphs))

DOCX loaded successfully
Total paragraphs in document: 436


In [ ]:
def is_heading(line):
    line = line.strip()
    if len(line) < 6:
        return False

    if re.fullmatch(r"[A-Z0-9/ &\-\(\)\.,]+", line) and sum(c.isalpha() for c in line) >= 5:
        return True

    return False

In [ ]:
# Show first 20 paragraphs and whether they look like headings
for i in range(20):
    text = doc.paragraphs[i].text.strip()
    if text:
        print(i, "| HEADING" if is_heading(text) else "| text", "->", text[:80])

0 | HEADING -> ADMISSIONS FREQUENTLY ASKED INQUIRY QUESTIONS TEMPLATES
3 | HEADING -> STATUS OF APPLICATION WHEN CUNYFIRST CHECKLIST STATES APPLICATION IS UNDER REVIE
5 | text -> Dear Applicant,
7 | text -> Thank you for your interest in Brooklyn College.
9 | text -> According to our records, your CUNYfirst application for [specific semester] is 
10 | text -> Please log in to your CUNYfirst account at https://home.cunyfirst.cuny.edu using
11 | text -> Once logged in:
12 | text -> Go to Student Center
13 | text -> Scroll down to the My Applications section
14 | text -> Click View Details for your Brooklyn College application
15 | text -> There, you can see which supporting documents have been received and which are s
16 | text -> Sincerely,
17 | text -> Office of Undergraduate Recruitment
18 | HEADING -> STATUS OF APPLICATION WHEN CUNYFIRST STATES APPLICATION IS COMPLETE AND BEING RE


In [ ]:
def split_docx_into_sections(doc):
    sections = []

    current_title = "UNTITLED"
    current_lines = []

    current_start = None
    last_idx = None

    for i, p in enumerate(doc.paragraphs):
        line = (p.text or "").strip()

        if line == "":
            if current_lines and current_lines[-1] != "":
                current_lines.append("")
            if current_start is None:
                current_start = i
            last_idx = i
            continue

        if current_start is None:
            current_start = i

        # new section
        if is_heading(line) and (not current_lines or len(current_lines) > 3):
            if current_lines:
                text = "\n".join(current_lines).strip()
                if text:
                    sections.append({
                        "title": current_title,
                        "text": text,
                        "start_idx": current_start,
                        "end_idx": last_idx if last_idx is not None else i - 1
                    })

            current_title = line
            current_lines = []
            current_start = i
            last_idx = i
        else:
            current_lines.append(line)
            last_idx = i

    # last section
    if current_lines:
        text = "\n".join(current_lines).strip()
        if text:
            sections.append({
                "title": current_title,
                "text": text,
                "start_idx": current_start,
                "end_idx": last_idx if last_idx is not None else current_start
            })

    return sections

sections = split_docx_into_sections(doc)

print("Sections found:", len(sections))
print("Example section:")
print("Title:", sections[1]["title"])
print("Range:", sections[1]["start_idx"], "to", sections[1]["end_idx"])
print("Preview:\n", sections[1]["text"][:300])

Sections found: 27
Example section:
Title: STATUS OF APPLICATION WHEN CUNYFIRST STATES APPLICATION IS COMPLETE AND BEING REVIEWED
Range: 18 to 26
Preview:
 Dear Applicant,

Thank you for your interest in Brooklyn College.
According to our records, your application is complete and has been processed. You will receive a decision regarding your application shortly. If you have any questions once the decision is issued, please visit the Office of Admission


In [ ]:
def tokenize(text):
    return re.findall(r"[a-zA-Z0-9']+", text.lower())

corpus = [tokenize(s["title"] + " " + s["text"]) for s in sections]
bm25 = BM25Okapi(corpus)

def retrieve_top_k(email_text, k=5):
    query = tokenize(email_text)
    scores = bm25.get_scores(query)
    top_idx = sorted(range(len(scores)), key=lambda i: scores[i], reverse=True)[:k]
    return [(i, scores[i], sections[i]) for i in top_idx]

print("BM25 ready")

BM25 ready


In [ ]:
def answer_from_doc_bm25_only(email_text, top_k=5):
    top = retrieve_top_k(email_text, k=top_k)
    best = top[0][2]  # best scoring section
    return best

test_email = "Hi, my CUNYfirst checklist says my application is under review. What does that mean?"
best_section = answer_from_doc_bm25_only(test_email)

print("Chosen Title:", best_section["title"])
print("Paragraph range:", best_section["start_idx"], "to", best_section["end_idx"])
print("\nPreview:\n", best_section["text"][:500])

Chosen Title: STATUS OF APPLICATION WHEN CUNYFIRST CHECKLIST STATES APPLICATION IS UNDER REVIEW
Paragraph range: 3 to 17

Preview:
 Dear Applicant,

Thank you for your interest in Brooklyn College.

According to our records, your CUNYfirst application for [specific semester] is currently under review. Please note that your application may be missing required supporting documents. If you have already sent your transcript, allow 2-3 weeks for confirmation of receipt to appear in the “Action” column of the supporting documents section.
Please log in to your CUNYfirst account at https://home.cunyfirst.cuny.edu using your usernam


In [ ]:
from copy import deepcopy
from docx import Document

def export_section_as_docx(docx_path, section, out_path="chosen_template.docx"):
    src = Document(docx_path)

    # Create output doc using the same file
    out = Document(docx_path)

    body = out.element.body

    # Remove all existing content
    for child in list(body):
        body.remove(child)

    start_i = section["start_idx"]
    end_i = section["end_idx"]

    # Copy exact paragraph
    for i in range(start_i, end_i + 1):
        p = src.paragraphs[i]._p
        body.append(deepcopy(p))

    out.save(out_path)
    return out_path

In [ ]:
out_file = export_section_as_docx(DOCX_PATH, best_section, "chosen_template.docx")
print("Saved:", out_file)

from google.colab import files
files.download(out_file)

Saved: chosen_template.docx


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
print("Paste the full email below.")
print("When finished, press ENTER twice.\n")

lines = []
while True:
    line = input()
    if line == "":
        break
    lines.append(line)

email_input = "\n".join(lines)

print("\n--- Processing Email ---\n")

chosen_section = answer_from_doc_bm25_only(email_input)

print("Chosen Title:", chosen_section["title"])
print("Range:", chosen_section["start_idx"], "to", chosen_section["end_idx"])

out_file = export_section_as_docx(DOCX_PATH, chosen_section, "chosen_template.docx")
print("\nSaved formatted DOCX as:", out_file)

from google.colab import files
files.download(out_file)

Paste the full email below.
When finished, press ENTER twice.

What is the Toefl score for Brooklyn college?


--- Processing Email ---

Chosen Title: EXEMPTIONS TO TOEFL/IELTS/PTE
Range: 174 to 205

Saved formatted DOCX as: chosen_template.docx


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>